In [ ]:
# importing libraries

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from google.colab import drive
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers,models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import Callback, EarlyStopping,ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import Model
from tensorflow.keras.layers.experimental import preprocessing
from pathlib import Path
import os.path
from sklearn.metrics import classification_report, confusion_matrix
import itertools

In [ ]:
# Mount Google Drive to access the dataset
drive.mount('/content/drive')

# Set dataset directory (update with your dataset path in Google Drive)
dataset_dir = '/content/drive/My Drive/EnhancedModel_MobileNetV2'

Mounted at /content/drive


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,                # Normalize pixel values
    rotation_range=15,             # Reduce rotation to realistic range for leaves
    width_shift_range=0.1,         # Slight horizontal shift
    height_shift_range=0.1,        # Slight vertical shift
    shear_range=0.1,               # Reduced shear for natural appearance
    zoom_range=0.1,                # Subtle zoom variations
    horizontal_flip=True,          # Flipping for mirrored leaf positions
    brightness_range=[0.8, 1.2],   # Adjust brightness to simulate lighting conditions
    fill_mode='nearest',           # Fill pixels with nearest values
    validation_split=0.2           # 20% of data for validation (adjust based on dataset size)
)

In [ ]:
# Training and validation generators
train_generator = train_datagen.flow_from_directory(
    '/content/drive/My Drive/EnhancedModel_MobileNetV2/train', # Path to dataset directory
    target_size=(224, 224),        # Resize images to match model input size
    batch_size=32,                 # Adjust batch size based on GPU memory
    class_mode='categorical',      # Use 'categorical' for multi-class classification
    subset='training'              # Training subset
)

Found 2466 images belonging to 4 classes.


In [ ]:
validation_generator = train_datagen.flow_from_directory(
    '/content/drive/My Drive/EnhancedModel_MobileNetV2/val', # Same dataset path
    target_size=(224, 224),        # Same size as training
    batch_size=32,                 # Match training batch size
    class_mode='categorical',      # Ensure consistent class mode
    subset='validation'            # Validation subset
)

Found 152 images belonging to 4 classes.


In [ ]:
# The base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
# Freeze the base model layers so they don't get trained
base_model.trainable = False

In [ ]:
# Build the final model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),  # Add dropout to prevent overfitting
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 classes: Blight, Common Rust, Gray Leaf Spot, Healthy
])

from tensorflow.keras.metrics import Precision, Recall

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy', Precision(name='precision'), Recall(name='recall')]
)


In [ ]:
from math import ceil
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping to monitor validation loss and stop training when it stops improving
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,         # Stop after 5 epochs with no improvement
    restore_best_weights=True
)

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    steps_per_epoch=ceil(train_generator.samples / train_generator.batch_size),
    validation_steps=ceil(validation_generator.samples / validation_generator.batch_size),
    verbose=1,
    callbacks=[early_stopping]  # Add early stopping
)

Epoch 1/10
78/78 [==============================] - 499s 6s/step - loss: 0.4766 - accuracy: 0.8491 - precision: 0.8580 - recall: 0.8427 - val_loss: 0.1730 - val_accuracy: 0.9408 - val_precision: 0.9470 - val_recall: 0.9408
Epoch 2/10
78/78 [==============================] - 156s 2s/step - loss: 0.3139 - accuracy: 0.8925 - precision: 0.8958 - recall: 0.8889 - val_loss: 0.1985 - val_accuracy: 0.9145 - val_precision: 0.9205 - val_recall: 0.9145
Epoch 3/10
78/78 [==============================] - 152s 2s/step - loss: 0.2440 - accuracy: 0.9079 - precision: 0.9144 - recall: 0.9051 - val_loss: 0.2048 - val_accuracy: 0.9342 - val_precision: 0.9333 - val_recall: 0.9211
Epoch 4/10
78/78 [==============================] - 153s 2s/step - loss: 0.2285 - accuracy: 0.9108 - precision: 0.9142 - recall: 0.9075 - val_loss: 0.1971 - val_accuracy: 0.9211 - val_precision: 0.9211 - val_recall: 0.9211
Epoch 5/10
78/78 [==============================] - 153s 2s/step - loss: 0.2218 - accuracy: 0.9116 - precisi

In [ ]:
# Save the trained model
model.save('/content/drive/My Drive/final_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
